# Overview
- officialなbssidを特定し、プロットしてみる

In [1]:
import subprocess
cmd = "git rev-parse --short HEAD"
hash = subprocess.check_output(cmd.split()).strip().decode('utf-8')
print(hash)

dec5b94


# Const

In [2]:
NB = '026'
DIR_TRAIN = './../data_ignore/input/train/'
DIR_TEST = './../data_ignore/input/test/'
DIR_WIFI = './../data_ignore/input/wifi/'
PATH_SUB = './../data_ignore/input/sample_submission.csv'
PATH_99_SUB = './../data/input/floor_99per_acc_sub.csv'
DIR_SAVE_IGNORE = f'./../data_ignore/nb/{NB}/'
DIR_SAVE = f'./../data/nb/{NB}/'

In [3]:
config_str = '''
globals:
    seed: 5713
    device: cuda
    n_label: 24
    n_splits: 5
    random_sate: 42
    lr: 0.001
    patience: 10
    epoch: 100
    batch_size: 512
    skip_evaluate_num: 5
    num_feats: 20
    t_mux: 10
'''

# Import everything I need:)

In [4]:
import os
import time
import yaml
import types
import random
import pickle
import builtins
import numpy as np
import pandas as pd
import seaborn as sns
from icecream import ic
import matplotlib.pyplot as plt
from dataclasses import dataclass
# from tqdm import tqdm
from fastprogress import progress_bar, master_bar
from glob import glob
from loguru import logger
from collections import OrderedDict

# sklearn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import KFold

# pytorch
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F

# Function

In [5]:
def imports():
    for name, val in globals().items():
        # module imports
        if isinstance(val, types.ModuleType):
            yield name, val

            # functions / callables
        if hasattr(val, '__call__'):
            yield name, val


def noglobal(f):
    '''
    ref: https://gist.github.com/raven38/4e4c3c7a179283c441f575d6e375510c
    '''
    return types.FunctionType(f.__code__,
                              dict(imports()),
                              f.__name__,
                              f.__defaults__,
                              f.__closure__
                              )


def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [6]:
@dataclass
class ReadData:
    acce: np.ndarray
    acce_uncali: np.ndarray
    gyro: np.ndarray
    gyro_uncali: np.ndarray
    magn: np.ndarray
    magn_uncali: np.ndarray
    ahrs: np.ndarray
    wifi: np.ndarray
    ibeacon: np.ndarray
    waypoint: np.ndarray


def read_data_file(data_filename):
    acce = []
    acce_uncali = []
    gyro = []
    gyro_uncali = []
    magn = []
    magn_uncali = []
    ahrs = []
    wifi = []
    ibeacon = []
    waypoint = []

    with open(data_filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for line_data in lines:
        line_data = line_data.strip()
        if not line_data or line_data[0] == '#':
            continue

        line_data = line_data.split('\t')

        if line_data[1] == 'TYPE_ACCELEROMETER':
            acce.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_ACCELEROMETER_UNCALIBRATED':
            acce_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_GYROSCOPE':
            gyro.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_GYROSCOPE_UNCALIBRATED':
            gyro_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_MAGNETIC_FIELD':
            magn.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_MAGNETIC_FIELD_UNCALIBRATED':
            magn_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_ROTATION_VECTOR':
            ahrs.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_WIFI':
            sys_ts = line_data[0]
            ssid = line_data[2]
            bssid = line_data[3]
            rssi = line_data[4]
            lastseen_ts = line_data[6]
            wifi_data = [sys_ts, ssid, bssid, rssi, lastseen_ts]
            wifi.append(wifi_data)
            continue

        if line_data[1] == 'TYPE_BEACON':
            ts = line_data[0]
            uuid = line_data[2]
            major = line_data[3]
            minor = line_data[4]
            rssi = line_data[6]
            ibeacon_data = [ts, '_'.join([uuid, major, minor]), rssi]
            ibeacon.append(ibeacon_data)
            continue

        if line_data[1] == 'TYPE_WAYPOINT':
            waypoint.append([int(line_data[0]), float(line_data[2]), float(line_data[3])])

    acce = np.array(acce)
    acce_uncali = np.array(acce_uncali)
    gyro = np.array(gyro)
    gyro_uncali = np.array(gyro_uncali)
    magn = np.array(magn)
    magn_uncali = np.array(magn_uncali)
    ahrs = np.array(ahrs)
    wifi = np.array(wifi)
    ibeacon = np.array(ibeacon)
    waypoint = np.array(waypoint)

    return ReadData(acce, acce_uncali, gyro, gyro_uncali, magn, magn_uncali, ahrs, wifi, ibeacon, waypoint)

# Preparation

load config

In [7]:
config = yaml.safe_load(config_str)
config

{'globals': {'seed': 5713,
  'device': 'cuda',
  'n_label': 24,
  'n_splits': 5,
  'random_sate': 42,
  'lr': 0.001,
  'patience': 10,
  'epoch': 100,
  'batch_size': 512,
  'skip_evaluate_num': 5,
  'num_feats': 20,
  't_mux': 10}}

<br>

set

In [8]:
seed_everything(config['globals']['seed'])
pd.set_option('display.max_rows', 500)

if not os.path.exists(DIR_SAVE_IGNORE):
    os.makedirs(DIR_SAVE_IGNORE)
if not os.path.exists(DIR_SAVE):
    os.makedirs(DIR_SAVE)

<br>

load dataset

In [9]:
with open(f'{DIR_WIFI}train_all.pkl', 'rb') as f:
    df_train = pickle.load( f)
with open(f'{DIR_WIFI}test_all.pkl', 'rb') as f:
    df_test = pickle.load( f)

sample_submission = pd.read_csv(PATH_SUB)

# EDA

In [20]:
path_list = sorted(glob('./../data_ignore/nb/004/*'))
path_list

['./../data_ignore/nb/004/nb004_wifi_5a0546857ecc773753327266.csv',
 './../data_ignore/nb/004/nb004_wifi_5c3c44b80379370013e0fd2b.csv',
 './../data_ignore/nb/004/nb004_wifi_5d27075f03f801723c2e360f.csv',
 './../data_ignore/nb/004/nb004_wifi_5d27096c03f801723c31e5e0.csv',
 './../data_ignore/nb/004/nb004_wifi_5d27097f03f801723c320d97.csv',
 './../data_ignore/nb/004/nb004_wifi_5d27099f03f801723c32511d.csv',
 './../data_ignore/nb/004/nb004_wifi_5d2709a003f801723c3251bf.csv',
 './../data_ignore/nb/004/nb004_wifi_5d2709b303f801723c327472.csv',
 './../data_ignore/nb/004/nb004_wifi_5d2709bb03f801723c32852c.csv',
 './../data_ignore/nb/004/nb004_wifi_5d2709c303f801723c3299ee.csv',
 './../data_ignore/nb/004/nb004_wifi_5d2709d403f801723c32bd39.csv',
 './../data_ignore/nb/004/nb004_wifi_5d2709e003f801723c32d896.csv',
 './../data_ignore/nb/004/nb004_wifi_5da138274db8ce0c98bbd3d2.csv',
 './../data_ignore/nb/004/nb004_wifi_5da1382d4db8ce0c98bbe92e.csv',
 './../data_ignore/nb/004/nb004_wifi_5da138314db

In [11]:
n_row = 100
print('='*50)
print(f'各siteにつき、bssid {n_row}個だけ表示')
print('='*50)
for i_path, path in enumerate(path_list):
    site = path.split('_')[-1].split('.')[0]
    df = pd.read_csv(path).astype({'rssi': 'float16'})
    df_grp = df.groupby(['bssid', 'floor']).agg(cnt=('bssid', 'count'), rssi_median=('rssi', 'median'))
    
    print('-'*50)
    print(f'\t site = {site}')
    print('-'*50)
    display(df_grp.head(n_row).style.bar(subset=['cnt', 'rssi_median'], align='left', color=['red', 'green']))
    print('')

各siteにつき、bssid 100個だけ表示
--------------------------------------------------
	 site = 5a0546857ecc773753327266
--------------------------------------------------



--------------------------------------------------
	 site = 5c3c44b80379370013e0fd2b
--------------------------------------------------



--------------------------------------------------
	 site = 5d27075f03f801723c2e360f
--------------------------------------------------



--------------------------------------------------
	 site = 5d27096c03f801723c31e5e0
--------------------------------------------------



--------------------------------------------------
	 site = 5d27097f03f801723c320d97
--------------------------------------------------


...


--------------------------------------------------
	 site = 5da1389e4db8ce0c98bd0547
--------------------------------------------------



--------------------------------------------------
	 site = 5da138b74db8ce0c98bd4774
--------------------------------------------------



--------------------------------------------------
	 site = 5da958dd46f8266d0737457b
--------------------------------------------------



--------------------------------------------------
	 site = 5dbc1d84c1eb61796cf7c010
--------------------------------------------------



--------------------------------------------------
	 site = 5dc8cea7659e181adb076a3f
--------------------------------------------------


## これ大事 ssid 毎の bssid数

In [12]:
n_row = 15
for i_path, path in enumerate(path_list):
    site = path.split('_')[-1].split('.')[0]
    df = pd.read_csv(path).astype({'rssi': 'float16'})
    
    print('-'*50)
    print(f'\t site = {site}')
    print('-'*50)
    df_grp = df[['ssid ', 'bssid']].groupby('ssid ').nunique().sort_values('bssid', ascending=False)
    df_grp['change_rate'] = df[['ssid ', 'bssid']].groupby('ssid ').nunique().sort_values('bssid', ascending=False).pct_change(-1).values
    df_grp['change_rate_mean'] = df_grp['change_rate'].rolling(3, center=True, win_type='gaussian').mean(std=1 )
#     display(df_grp)
    display(df_grp.head(n_row).style.bar(subset=['bssid', 'change_rate', 'change_rate_mean'], align='left', color=['red', 'green']))
    print('')

--------------------------------------------------
	 site = 5a0546857ecc773753327266
--------------------------------------------------


,bssid,change_rate,change_rate_mean
ssid,,,
da39a3ee5e6b4b0d3255bfef95601890afd80709,543,0.491758,nan
7182afc4e5c212133d5d7d76eb3df6c24618302b,364,0.005525,0.138795
b7e6027447eb1f81327d66cfd3adbe557aabf26c,362,0.005556,0.008670
b6ffe5619e02871fcd04f61c9bb4b5c53a3f46b7,360,0.016949,0.117082
d839a45ebe64ab48b60a407d837fb01d3c0dfef9,354,0.393701,0.346296
78f32ce289cdaccfc83a01ef3cbf4af52942c063,254,0.597484,0.383152
b9f0208be00bd8b337be7f12e02e3a3ce846e22b,159,0.019231,1.934311
5731b8e08abc69d4c4d685c58164059207c93310,156,6.428571,3.115654
dbc0f004854457f59fb16ab863a3a1722cef553f,21,0.750000,2.100767



--------------------------------------------------
	 site = 5c3c44b80379370013e0fd2b
--------------------------------------------------


,bssid,change_rate,change_rate_mean
ssid,,,
44aebad0d3851fd527d792ef1af44d24f41b5799,921,0.112319,nan
e4e0e9d20827e770b65693ba9fdaefa403d5dfe4,828,1.670968,1.799055
da39a3ee5e6b4b0d3255bfef95601890afd80709,310,3.696970,2.371229
ca969a043e538dab9b1e3d3bf0e82b34c962e019,66,0.885714,1.539059
b8ea8cf3e81583f91a2d3f5253da09b1276d86ba,35,0.458333,0.449850
e46d2577d6f83617f3a8c416607f8fe1263c4483,24,0.000000,0.125615
9ee806540f418f1fffce5197aac0717792293d35,24,0.000000,0.054814
6ccdd75ff3326eb1517a988af5e5661f3bb4bdef,24,0.200000,0.273085
dbc0f004854457f59fb16ab863a3a1722cef553f,20,0.666667,0.356056



--------------------------------------------------
	 site = 5d27075f03f801723c2e360f
--------------------------------------------------


...

,bssid,change_rate,change_rate_mean
ssid,,,
da39a3ee5e6b4b0d3255bfef95601890afd80709,1465,1.474662,nan
90a13d9926e0f11cb22ac9bb4d245a64ed87411c,592,23.666667,11.110160
6cfcff54df27ca5baef9574cffb18326c7b5852d,24,0.043478,6.505937
5fb64a415514541555376c359dcf099886be5ff2,23,0.000000,0.024374
980fc4b857abe964c86ba499516fe46673dc4d68,23,0.045455,0.020539
ea1742a21aeaadc8b415d708b5eb5a47e4b780fe,22,0.000000,0.012458
f815eabb67168325255092b44a8a9861b18e1deb,22,0.000000,0.027407
9b96041fcdf03a9b1574dc75cc9d23caed9460a4,22,0.100000,0.075638
b8599529669658d73f1eba6606e3a81352e7fb80,20,0.111111,0.155919



--------------------------------------------------
	 site = 5dbc1d84c1eb61796cf7c010
--------------------------------------------------


,bssid,change_rate,change_rate_mean
ssid,,,
da39a3ee5e6b4b0d3255bfef95601890afd80709,1966,1.283391,nan
8f7d5669fb8464cd2eef926f246adaac8050b848,861,22.916667,10.881333
9f755fbf3a18018268b924f35ddeaab61072ed33,36,0.636364,6.581339
bbec6798f8898847f8f4cf2f77356ddba622aaa4,22,0.047619,0.195925
24461fb772d8b95f99df1bf82b72c050010d148e,21,0.000000,0.013051
3c2bfa511c1e8e363d08ea99efeb4b6a51d260f1,21,0.000000,0.000000
e75b5d9b518e620da0ee3d8382f2086778059dde,21,0.000000,0.000000
5c8cc1443c4b580c96f3b830c90e3d3cc257409e,21,0.000000,0.000000
b8fae28e6654aabc149345406e601e9024ee52c1,21,0.000000,0.205551



--------------------------------------------------
	 site = 5dc8cea7659e181adb076a3f
--------------------------------------------------


,bssid,change_rate,change_rate_mean
ssid,,,
da39a3ee5e6b4b0d3255bfef95601890afd80709,767,0.049248,nan
5d998a8668536c4f51004c25f474117fe9555f78,731,8.746667,4.218777
ca969a043e538dab9b1e3d3bf0e82b34c962e019,75,0.923077,2.821503
dc5c79cadaa151736d79671f33d77fafe7dcf106,39,0.026316,0.391371
b8ea8cf3e81583f91a2d3f5253da09b1276d86ba,38,0.461538,0.226727
f11066513ace56d9b2a0d049e0f186f2c5fabef5,26,0.040000,0.359907
75d4a6063cb86b0a7aecda5f00f4fb4eecc26f5e,25,0.785714,0.387080
ca91fc2f06ad6e4e122bcbf33d50bc25d24c6e92,14,0.076923,0.250098
dbc0f004854457f59fb16ab863a3a1722cef553f,13,0.000000,0.021082


# 紐づくbssidが多いssidを各階で抽出

In [13]:
n_ssid = 5
for i_path, path in enumerate(path_list):
    site = path.split('_')[-1].split('.')[0]
    df = pd.read_csv(path).astype({'rssi': 'float16'})
    ssids = df.groupby(['ssid '])[['bssid']].count().sort_values('bssid', ascending=False).index[:n_ssid].values
    
    print('-'*50)
    print(f'\t site = {site}')
    print(f'\t {df.floor.unique()}')
    print('-'*50)
    

    df_grp = df[['ssid ', 'bssid', 'floor']].groupby(['ssid ', 'floor']).nunique().sort_values('bssid', ascending=False).reset_index()
    df_grp['ssid'] = df_grp['ssid ']
#     ssids = df_grp['ssid'].unique()[:n_ssid]
    df_grp = df_grp.query('ssid in @ssids')
    display(df_grp.drop(['ssid'], axis=1).sort_values(['ssid ', 'bssid', 'floor'], ascending=False).set_index(['ssid ', 'floor']))
    print('')

--------------------------------------------------
	 site = 5a0546857ecc773753327266
	 ['B1' 'F1' 'F2' 'F3' 'F4']
--------------------------------------------------


bssid
ssid                                     floor       
da39a3ee5e6b4b0d3255bfef95601890afd80709 F1       376
                                         F2       296
                                         B1       241
                                         F4        52
                                         F3        45
d839a45ebe64ab48b60a407d837fb01d3c0dfef9 F1       227
                                         F2       217
                                         F3       211
                                         F4       210
                                         B1       149
b7e6027447eb1f81327d66cfd3adbe557aabf26c F1       227
                                         F2       220
                                         F4       215
                                         F3       213
                                         B1       158
b6ffe5619e02871fcd04f61c9bb4b5c53a3f46b7 F1       226
                                         F2       219
                                         F4       212
                                         F3       212
                                         B1       151
7182afc4e5c212133d5d7d76eb3df6c24618302b F1       232
                                         F2       223
                                         F4       212
                                         F3       210
                                         B1       157


--------------------------------------------------
	 site = 5c3c44b80379370013e0fd2b
	 ['B1' 'F1' 'F2' 'F3' 'F4' 'F5']
--------------------------------------------------


bssid
ssid                                     floor       
e4e0e9d20827e770b65693ba9fdaefa403d5dfe4 F2       706
                                         B1       548
                                         F1       453
                                         F4       214
                                         F5       169
                                         F3       164
da39a3ee5e6b4b0d3255bfef95601890afd80709 F1       261
                                         F2       212
                                         B1       112
                                         F3        99
                                         F4        83
                                         F5        50
ca969a043e538dab9b1e3d3bf0e82b34c962e019 F1        60
                                         F2        46
                                         F3        19
                                         F4        15
                                         B1        10
                                         F5         5
b8ea8cf3e81583f91a2d3f5253da09b1276d86ba F1        32
                                         F2        27
                                         F4        12
                                         F3        12
                                         F5         5
                                         B1         4
44aebad0d3851fd527d792ef1af44d24f41b5799 F1       775
                                         F2       732
                                         F3       699
                                         B1       648
                                         F4       541
                                         F5       367


--------------------------------------------------
	 site = 5d27075f03f801723c2e360f
	 ['B1' 'F1' 'F2' 'F3' 'F4' 'F5' 'F6' 'F7']
--------------------------------------------------


...

bssid
ssid                                     floor       
da39a3ee5e6b4b0d3255bfef95601890afd80709 F1       727
                                         F3       710
                                         F2       697
                                         F4       622
                                         F5       552
                                         B1       530
                                         F6       455
                                         F7       190
b81101927a13e44bd6788592b492ba0e88b50967 F6        10
                                         F7         8
                                         F5         8
                                         F4         7
                                         F3         7
                                         F1         4
                                         F2         3
                                         B1         2
9e2a809aad37dae3058a3e65449dc67ffedcd7e8 F2         8
                                         F1         8
                                         B1         7
                                         F3         6
                                         F5         4
                                         F4         4
                                         F6         3
90a13d9926e0f11cb22ac9bb4d245a64ed87411c F3       314
                                         F2       293
                                         F1       284
                                         F4       275
                                         F5       249
                                         B1       211
                                         F6       210
                                         F7        79
6cfcff54df27ca5baef9574cffb18326c7b5852d F7        18
                                         F6        14
                                         F5        14
                                         F1        11
                                         F4        10
                                         F3         8
                                         F2         8
                                         B1         4


--------------------------------------------------
	 site = 5dbc1d84c1eb61796cf7c010
	 ['B1' 'F2' 'F3' 'F4' 'F5' 'F6' 'F7' 'F8']
--------------------------------------------------


bssid
ssid                                     floor       
da39a3ee5e6b4b0d3255bfef95601890afd80709 F2      1143
                                         F4      1139
                                         F3      1111
                                         F5      1015
                                         F6       982
                                         F7       815
                                         B1       554
                                         F8       462
9f755fbf3a18018268b924f35ddeaab61072ed33 F7        33
                                         F8        26
                                         F6        14
                                         F5        12
                                         F2         7
                                         F4         6
                                         F3         5
8f7d5669fb8464cd2eef926f246adaac8050b848 F4       565
                                         F3       535
                                         F2       532
                                         F5       518
                                         F6       494
                                         F7       430
                                         F8       252
                                         B1       243
5eddd2b748b50cbfa67d90f0ef019e6e822e38f5 F8         6
                                         F7         6
                                         F6         6
                                         F5         5
                                         F4         4
                                         F3         4
                                         F2         4
37d2ae1153841a0fb2002864a770bf7d2e9492b8 F8         8
                                         F7         8
                                         F6         8
                                         F5         7
                                         F4         5
                                         F3         4
                                         F2         3


--------------------------------------------------
	 site = 5dc8cea7659e181adb076a3f
	 ['B1' 'F1' 'F2' 'F3' 'F4' 'F5' 'F6' 'F7']
--------------------------------------------------


bssid
ssid                                     floor       
da39a3ee5e6b4b0d3255bfef95601890afd80709 F2       454
                                         F1       419
                                         F3       263
                                         F4       184
                                         F5       173
                                         F6       109
                                         F7        88
                                         B1        81
ca969a043e538dab9b1e3d3bf0e82b34c962e019 F1        62
                                         F2        48
                                         F3        22
                                         F4         7
                                         F5         6
                                         B1         3
                                         F6         2
b8ea8cf3e81583f91a2d3f5253da09b1276d86ba F1        30
                                         F2        27
                                         F3        13
                                         F4         7
                                         F5         6
                                         F6         1
a39f7616b9bb60d3c3fd2bcdcc6e09f4ce6a8cb1 F6         8
                                         F5         8
                                         F4         8
                                         F2         6
                                         F1         6
                                         F3         5
5d998a8668536c4f51004c25f474117fe9555f78 F3       512
                                         F2       495
                                         F1       484
                                         F4       448
                                         F5       420
                                         F6       372
                                         B1        88
                                         F7        47

# 質の良いbssidを選定
- 質の良い: bssid

In [21]:
path = './../data/nb/013/nb013_bssid_position_*.csv'
path_list_posi = sorted(glob(path))

,ssid,bssid,bssid_x,bssid_y,n_samples,n_samples_rssi_over_m50,n_samples_rssi_over_m55,n_samples_rssi_over_m60,n_samples_rssi_over_m65,n_samples_rssi_over_m70,site,floor
0,da39a3ee5e6b4b0d3255bfef95601890afd80709,c08ad78a45798cfe176a42b35c7381ae602711c5,220.011148,157.668275,52,14,14,14,16,18,5a0546857ecc773753327266,B1
1,da39a3ee5e6b4b0d3255bfef95601890afd80709,ffc7c34369257431c7de9129094deb923bb3e3af,208.995536,156.881556,25,1,5,13,14,14,5a0546857ecc773753327266,B1
2,da39a3ee5e6b4b0d3255bfef95601890afd80709,24d178b2fe580b871c853757e2c4668b16bc3ffc,230.002040,153.508576,11,0,0,0,1,1,5a0546857ecc773753327266,B1
3,da39a3ee5e6b4b0d3255bfef95601890afd80709,23bfb8a3a2936f0536b7b8c1ee4a13706b8c18c6,204.576861,162.133758,47,0,1,4,4,17,5a0546857ecc773753327266,B1
4,da39a3ee5e6b4b0d3255bfef95601890afd80709,059ea3d13de011f91587f1d176599605274f8ee8,170.015288,167.586402,65,0,4,10,10,14,5a0546857ecc773753327266,B1


In [50]:
for i in range(len(path_list)):
    site = path.split('_')[-1].split('.')[0]
    print('-'*50)
    print(f'\t site = {site}')
    print(f'\t {df.floor.unique()}')
    print('-'*50)
    
    df_wifi = pd.read_csv(path_list[i])
    df_posi = pd.read_csv(path_list_posi[i])
    top_ssid = df_wifi.groupby(['ssid '])[['bssid']].count().sort_values('bssid', ascending=False).index[0]
    display(df_posi[df_posi.ssid==top_ssid][['bssid', 'n_samples']].sort_values('n_samples', ascending=False).head(100).style.bar())

--------------------------------------------------
	 site = *
	 ['B1' 'F1' 'F2' 'F3' 'F4']
--------------------------------------------------


,bssid,n_samples
5786,4cff1c8cfec27801ddc9a690ade87c57f1142ee0,192
3019,50354dce4504e2e95b814a2680e8f35d70ad810f,182
2945,0482b38bf4daa9243fff612a2594d8206fde8fd5,179
2986,6b6c054161e0f561d43041fbda8284626c95dd25,172
2947,d0f0c9fa52edde81962135fcf5e5b11e53ddd252,164
2966,c1729fc6f537c4e98cb041b1df671de29d808913,164
2915,4c855e83713c01e6a46a1669db047dca05320474,162
2949,2a8748d9e77634ce29b193eb6d3fed569755841e,162
2996,10cf16ad6534635699a05f3a6c344526b9d7d549,161
2965,963b140eb4a6b676ed10c75f6f0d3b8203958b35,158


--------------------------------------------------
	 site = *
	 ['B1' 'F1' 'F2' 'F3' 'F4']
--------------------------------------------------


,bssid,n_samples
181,c59474d3cdd705513791744271bd9b816bc82381,152
205,148bc9a8f7639b72f702c853c0f15d1e8b1708c1,152
226,7cdf384eec10405a8070a1aae8512ba88dccc18a,145
239,c3fd53a8407348b7b9c7a859ff6b7b3cd81cbf49,133
230,054e5a904789e1b21bd02af41ef2f9b46d89c7c8,133
227,c1dc6b624ad216d15bb930407465914e3deb1d3b,133
268,c4c0dbec4cdd8d29d98d9ae502263da3aa1b8c88,130
235,fb858eae1c32ee12b25c53c0bf7bd0b738796977,130
309,947d2cf6ebf4e849b2d3e4a855ae100cfa4e9b4f,130
270,7431359593da10cf88188fa63c80a2cacea645f8,127


--------------------------------------------------
	 site = *
	 ['B1' 'F1' 'F2' 'F3' 'F4']
--------------------------------------------------


...

,bssid,n_samples
1129,87fc05372182531550085ef2c4d9584beb85412f,144
1135,1ae1a7d1c63bc140afbad269331d9ab1e1140880,143
1126,96f523c8eb2514b46a50fc33cb867574759494da,133
1124,23b9bfb0364d704b654e3f29172d974bba9c602a,130
1178,a037061305187e1dd65ddf5cecb993b1e6f02e98,129
1162,f8bf0d87bb096f94783040765fc6c367aab0a884,129
1211,32f6748832132768023178d5637a03caecb8bc11,128
1128,780f8a8c78917c45f8202d11bec60dffb6006be5,127
1159,0a95042d5d7169d79324eec94081e9da84a2b577,125
1200,757001a47abb5a7eee6a473380cc705025fea33f,124


--------------------------------------------------
	 site = *
	 ['B1' 'F1' 'F2' 'F3' 'F4']
--------------------------------------------------


,bssid,n_samples
7483,af08d9b0add6418577ab0ade22a4f01e5c36f9db,207
2971,b094b601030229edf50c63ea0f7993717dadbae6,206
9229,313932df9d2c0c1ee184cb2bcb4ba0c8293b707e,203
2972,bc78f87d0471dff16391f82931b92aa614936591,203
9042,4ea8c5aeb259a3c856e8bfe8bb5a28014e1c3491,202
1261,7ffcaddf5587857644f5b64ad406efc2d383e34a,201
9183,785da2f9f6d9f4e1826102ea120e6d38ecdb3064,200
75,6ca691d7453dc31d38354fb85d029138db4e1718,199
9086,d62e509da2c7db42e0fce3f4f8936f5894c0ab4f,198
9185,b2d5a8bc00526c27f25e4857296066e9f16dc441,194


--------------------------------------------------
	 site = *
	 ['B1' 'F1' 'F2' 'F3' 'F4']
--------------------------------------------------


,bssid,n_samples
5832,95efe679d3fce0b8e9b86bef921fb8f825abf06d,197
340,d3d76a172ad37469252f6177cbbc877bb25ce9d7,192
5921,6c130d861527e6148a4e827e5e640c47ca3920ad,187
335,cb7f75899a58b16d82ecafa4ad9009dbf306f56f,183
4887,bc628ada0e1ddaa6722613317145a44b96e82073,173
5816,8b9f2bc2e71796c8557f6869e94a464474ccb493,172
324,d7d8a9e934c5aa448e2168806d74d07c5b25265c,168
5834,195eb3cc3f2b34a578f0df6082e204b8c333537b,167
339,bf39d2efcfd223df09cded2212b31e56b22ab3ee,166
4917,a8f5037bb878c0e77a2abbdbfd6c0d4a1e2b4dc0,166
